In [1]:
# from IPython.display import clear_output
import math


Q1: Choose and implement a formalism to represent all possible states or what is known 
as the state representation, including the initial state and the final state; the final state 
can be a winning / loosing state or a draw state (20%)

-#Initial States -#Final States (checkWhoWon)

Q2:Define and implement the list of possible actions at each time and, if needed, a 
validation function that will check whether or not a state / action is valid (20%)

-checkPoint,madeMill,canMove

Q3:The search tree may be huge for this game. Choose and implement an evaluation
function for a non-terminal state (15%)

-evalState

Q4. Implement the alpha-beta algorithm allowing the agent to play the game. Use the 
evaluation function, developed in question 3, to return the evaluation of a nonterminal state when the depth limit is reached (30%)

-Updated within minValue & maxValue functions

In [2]:
# Initial States - 

# Initializing the game state and board configuration
# creates an empty game state array with 16 slots
gameState = [' ' for _ in range(16)]

# A replect between pointIndex and pointPosition
points = ['a4', 'c4', 'e4','b3', 'c3', 'd3', 'a2', 'b2', 'd2', 'e2', 'b1', 'c1', 'd1', 'a0', 'c0', 'e0']

#setting up connections: key is the point, value is the list of the connected points
connections = {
    0: [1, 6],
    1: [0, 2, 4],
    2: [1, 9],
    3: [4, 7],
    4: [1, 3, 5],
    5: [4, 8],
    6: [0, 7, 13],
    7: [3, 6, 10],
    8: [5, 9, 12],
    9: [2, 8, 15],
    10: [7, 11],
    11: [10, 12, 14],
    12: [8, 11],
    13: [6, 14],
    14: [11, 13, 15],
    15: [9, 14],
}

# Defining the list of the mill:
mills = [[0, 1, 2],
         [3, 4, 5],
         [10, 11, 12],
         [13, 14, 15],
         [0, 6, 13],
         [3, 7, 10],
         [5, 8, 12],
         [2, 9, 15]]

# Setting up players and game parameters
# Initializing player turns, A for the agent(initial player), H for the Human (move after)
PlayerTurn = 'A'
Turn = 0
MaxDepth = 2 #depth influence the actions number (n*(n-1)*(n-2)*(n-depth+1)) choose the best depth

#identify a player class
class generatePlayer:
    def __init__(self, name, piece, board):
        self.name = name
        self.piece = piece
        self.board = board
        
playerA = generatePlayer('A', 6, 0)
playerH = generatePlayer('H', 6, 0)

# Core game logic functions:
# Evaluating the board state to see if it's reached the terminal(win,lose,draw)
def evalState(cur, player):
    evaluation = 0
    opponent = getOpponent(player)

    for mill in mills:
        playerCount = sum(cur[idx] == player.name for idx in mill)
        opponentCount = sum(cur[idx] == opponent.name for idx in mill)
        emptyCount = sum(cur[idx] == ' ' for idx in mill)

        if playerCount == 3:
            # Player has formed a mill (highest priority)
            evaluation += 10
        elif playerCount == 2 and emptyCount == 1:
            # Player has a potential mill
            evaluation += 5

        if opponentCount == 2 and emptyCount == 1:
            # Opponent has a potential mill (try to block)
            evaluation -= 5
        elif opponentCount == 3:
            evaluation -= 10
            
    return evaluation

def madeMill(cur, player, point):
    global mills
    for mill in mills:
        if point in mill:
            if all(cur[x] == player.name for x in mill):
                return True
    return False

#Minimax function deciding next move for AI player
#Defining possible actions:
def minimaxDecision(cur, player, depth, mode):
    bestValue = -math.inf
    alpha = -math.inf
    beta = math.inf
    bestPoint = None
    if mode == 'place': #when a player has pieces to place on the board
        for i in range(len(cur)):
            if cur[i] == ' ':
                if madeMill(cur, player, i):
                    value = 20
                else:                    
                    nextState = cur
                    nextState[i] = player.name  
                    value = minValue(nextState, getOpponent(player), depth - 1, alpha, beta)
                    nextState[i] = ' '  
                if value > bestValue:
                    bestValue = value
                    bestPoint = i
                    alpha = max(alpha, bestValue)
    elif mode == 'move': #when a player moves a piece from one point to another
        for i in range(len(cur)):
            if cur[i] == player.name:
                for movePoint in connections[i]:
                    if cur[movePoint] == ' ':
                        if madeMill(cur, player, movePoint):
                            value = 20
                        else:
                            nextState = cur
                            # Make the move
                            nextState[i] = ' '
                            nextState[movePoint] = player.name
                            value = minValue(nextState, getOpponent(player), depth - 1, alpha, beta)
                            # Undo the move
                            nextState[movePoint] = ' '
                            nextState[i] = player.name                
                        if value > bestValue:
                            bestValue = value
                            bestPoint = (movePoint, i)  # Store the source and destination indices
                            alpha = max(alpha, bestValue)
    elif mode == 'remove':#when a player forms a mill and can remove an opponent's piece.
        for i in range(len(cur)):
            if cur[i] == getOpponent(player).name:
                nextState = cur
                nextState[i] = ' '  # Remove opponent's piece
                value = minValue(nextState, getOpponent(player), depth - 1, alpha, beta)
                nextState[i] = getOpponent(player).name  # Undo removal
                if value > bestValue:
                    bestValue = value
                    bestPoint = i
                    alpha = max(alpha, bestValue)
    # print(mode, bestValue)
    return bestPoint

#updated alpha beta within minValue and maxValue
def minValue(cur, player, depth, alpha, beta):
    if depth <= 0:
        return evalState(cur, player)

    minValue = math.inf
    for i in range(len(cur)):
        if cur[i] == ' ':
            nextState = cur
            nextState[i] = player.name
            if madeMill(nextState,player, i):
                nextState[i] = ' '
                return -10
            else:
                value = maxValue(nextState, getOpponent(player), depth - 1, alpha, beta)
                nextState[i] = ' '
    
                if value < minValue:
                    minValue = value
                    if minValue <= alpha:
                        break  # Alpha cut-off
                    beta = min(beta, minValue)

    return minValue

def maxValue(cur, player, depth, alpha, beta):
    if depth <= 0:
        return evalState(cur, player)

    maxValue = -math.inf
    for i in range(len(cur)):
        if cur[i] == ' ':
            nextState = cur
            nextState[i] = player.name
            if madeMill(nextState,player, i):
                nextState[i] = ' '
                return 10
            else:
                value = minValue(nextState, getOpponent(player), depth - 1, alpha, beta)
                nextState[i] = ' '
    
                if value > maxValue:
                    maxValue = value
                    if maxValue >= beta:
                        break  # Beta cut-off
                    alpha = max(alpha, maxValue)

    return maxValue

# Utility and game flow functions
# change the turn of the player
def changePlayerTurn():
    global PlayerTurn, Turn
    if PlayerTurn == 'A':
        PlayerTurn = 'H'
    else:
        PlayerTurn = 'A'

# return to another player       
def getOpponent(player):
    if player.name == 'A':
        return playerH
    else:
        return playerA
    
def printGameBoard(cur):
    """Print the board in a human-readable format with lines and newlines."""
    # Clear the output to maintain a clean board state (p.s. using wait True often made input command disappear in this project）
#     clear_output(wait=False) 
    # Use a simple text representation for the board
    lines = [
        " 4 {}-----------{}-----------{}".format(cur[0], cur[1], cur[2]),
        "   |           |           |",
        " 3 |     {}-----{}-----{}     |".format(cur[3], cur[4], cur[5]),
        "   |     |           |     |",
        " 2 {}-----{}           {}-----{}".format(cur[6], cur[7], cur[8], cur[9]),
        "   |     |           |     |",
        " 1 |     {}-----{}-----{}     |".format(cur[10], cur[11], cur[12]),
        "   |           |           |",
        " 0 {}-----------{}-----------{}".format(cur[13], cur[14], cur[15]),
        "   a     b     c     d     e"
    ]    
    # Print the board
    for line in lines:
        print(line)
#     print("\nTurn:", "Agent" if PlayerTurn == 'A' else "Human")

# Check if the current player can make a move
def canMove(cur, player):
    """Check if the current player can make a move."""
    for i, v in enumerate(cur):
        if v == player.name:
            for conPoint in connections[i]:
#                 print(cur[conPoint])
                if cur[conPoint] == ' ':
                    return True                    
    return False




In [3]:
#Final State:
def checkWhoWon(cur):
    global playerA, playerH
    """Check the current state of the game for a win or a draw."""
    # If either player has less than 3 pieces on the board or cannot move, they lose
    if playerA.piece == 0 and playerA.board < 3:
        if not canMove(cur, playerA):
            print('Human wins')
            return True
    if playerH.piece == 0 and playerH.board < 3:
        if not canMove(cur, playerH):
            print('Agent wins')
            return True

    # If all pieces are placed and no one has won, check for a draw
    if playerA.piece == 0 and playerH.piece == 0 and playerA.board >= 3 and  playerH.board >= 3:
        # Optionally, check for repeated moves leading to a draw
        # Not implemented in this snippet
        pass

    # No win or draw detected, the game continues, once return True, one player win and the game end
    return False


# checkPoint that HUMAN player behaviours available and return
def checkPoint(mode, player, playerInput, choosePoint = None):
    global points
    point = None
    for i,v in enumerate(points):
        if mode == 'place' and v == playerInput and gameState[i] == ' ':
            point = i
        if mode == 'choose' and v == playerInput and gameState[i] == player.name and any(gameState[j] == ' ' for j in connections[i]):
            point = i
        if mode == 'move' and v == playerInput and gameState[i] == ' ' and choosePoint in connections[i]:
            point = i
        if mode == 'remove' and v == playerInput and gameState[i] == getOpponent(player).name:
            point = i
    if point == None:
        print('please input a right point!!!')
    return point

# a teststate for checkwhowon
testState = ['A',    'A',    'H',
                 'A','A','H',
             'A','A',    ' ',' ', 
                 'H',' ','H', 
             ' ',    'H',    ' ']


In [4]:
playerA.name

'A'

In [5]:
statePoint = [i for i in range(16)] 
printGameBoard(statePoint)

 4 0-----------1-----------2
   |           |           |
 3 |     3-----4-----5     |
   |     |           |     |
 2 6-----7           8-----9
   |     |           |     |
 1 |     10-----11-----12     |
   |           |           |
 0 13-----------14-----------15
   a     b     c     d     e


In [6]:
printGameBoard(testState)

 4 A-----------A-----------H
   |           |           |
 3 |     A-----A-----H     |
   |     |           |     |
 2 A-----A            ----- 
   |     |           |     |
 1 |     H----- -----H     |
   |           |           |
 0  -----------H----------- 
   a     b     c     d     e


In [7]:
canMove(testState, playerA)

True

In [8]:
checkWhoWon(testState)

False

Q5. Implement the main function that will allow the AI agent to play against a human 
being (15%)

-while(True): connect loop for the game

In [ ]:
# Game Start
PlayerTurn = 'H'
Turn = 0
# gameState = testState
# playerH.piece = 1
# playerH.board = 5
# playerA.piece = 0
# playerA.board = 6

printGameBoard(gameState)

while(True):
    putPoint = None
    choosePoint = None
    removePoint = None
    Turn = Turn + 1
    
    if (PlayerTurn == 'A'):     
        if playerA.piece > 0:
            bestPoint = minimaxDecision(gameState, playerA, MaxDepth, 'place')
            print('agent placed piece at '+ points[bestPoint] +' point')
            gameState[bestPoint] = playerA.name
            playerA.piece -= 1
            playerA.board += 1
        else:
            bestPoint, moveFrom  = minimaxDecision(gameState, playerA, MaxDepth, 'move')
            print('agent moved from {} to {}'.format(points[moveFrom], points[bestPoint]))
            gameState[moveFrom] = ' '
            gameState[bestPoint] = playerA.name
        printGameBoard(gameState)
        if madeMill(gameState, playerA, bestPoint):           
            removePoint = minimaxDecision(gameState, playerA, MaxDepth, 'remove')
            print('agent removed your '+ points[removePoint] +' point')
            gameState[removePoint] = ' '
            playerH.board = playerA.board - 1
            printGameBoard(gameState)    
        if (checkWhoWon(gameState)):
            break
        changePlayerTurn()
    else:
        if playerH.piece > 0:
            while(putPoint == None):
                playerInput = input('you have {} piece in your hand, choose the point to put a piece: '.format(playerH.piece))
                putPoint = checkPoint('place', playerH, playerInput)
            gameState[putPoint] = PlayerTurn
            playerH.piece = playerH.piece - 1
            playerH.board = playerH.board + 1
        else:
            while(choosePoint == None):
                playerInput = input('choose your piece to move: ')
                choosePoint = checkPoint('choose', playerH, playerInput)
            while(putPoint == None):
                playerInput = input('move to: ')
                putPoint = checkPoint('move', playerH, playerInput, choosePoint)
            gameState[choosePoint] = ' '
            gameState[putPoint] = PlayerTurn
        printGameBoard(gameState)
        if madeMill(gameState, playerH, putPoint):           
            while(removePoint == None):
                playerInput = input('remove a piece of your opponent: ')
                removePoint = checkPoint('remove', playerH, playerInput)
            gameState[removePoint] = ' '
            playerA.board = playerA.board - 1
            printGameBoard(gameState)    
        if (checkWhoWon(gameState)):
            break
        changePlayerTurn()

 4  ----------- ----------- 
   |           |           |
 3 |      ----- -----      |
   |     |           |     |
 2  -----             ----- 
   |     |           |     |
 1 |      ----- -----      |
   |           |           |
 0  ----------- ----------- 
   a     b     c     d     e


Q6. This question is optional / Bonus: Implement the Monte-Carlo Tree Search algorithm, 
and create a function allowing the two agents to play against each other (20%)

- Tried on another notebook but gave up eventrally

In [ ]:
import time
import random as rnd

from IPython.display import clear_output